# Wildfire Hazard Potential for the United States (270-m), version 2023 (4th Edition)

Dataset source: https://www.fs.usda.gov/rds/archive/products/RDS-2015-0047-4/_fileindex_RDS-2015-0047-4.html

Documentation: https://firelab.org/sites/default/files/2024-02/whp2023_cls_conus_metadata.pdf

https://data-usfs.hub.arcgis.com/datasets/usfs::wildfire-hazard-potential-version-2023-classified-image-service/explore?location=35.134247%2C-116.251988%2C6.95

https://www.fs.usda.gov/rds/archive/catalog/RDS-2015-0047-4


## Data dictionary

https://www.fs.usda.gov/rds/archive/products/RDS-2015-0047-4/_metadata_RDS-2015-0047-4.html

In [63]:
import json
import os
import random

from glob import glob

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import rioxarray
import xarray as xr

from rasterio.enums import Resampling
from rasterio.errors import WindowError
from rasterio.features import geometry_mask
from rasterio.mask import mask
from rasterio.windows import Window

from shapely.geometry import Point, box

In [ ]:
whp_filename = '../../wildfire-risk/data/Data-2/whp2023_GeoTIF/whp2023_cnt_conus.tif'

with rasterio.open(whp_filename) as src2:
    bounds = src2.bounds
    bbox = box(*bounds)
    geo = gpd.GeoDataFrame({'geometry': [bbox]}, crs=src2.crs)
    geo_json = [json.loads(geo.to_json())['features'][0]['geometry']]


In [ ]:
# Path to the folder containing .tif files
folder_path = "sim2real_layouts"

# Get a list of all .tif files in the folder
tif_files = glob(os.path.join(folder_path, "*.tif"))

layout_list = []

# Open each .tif file
for tif_file in tif_files:
    id = tif_file.split('/')[1].split('_E')[0]
    print(f"Processing {id}...")

    output_path = os.path.join("risk_layouts", f"whpi2023_cnt_{id}.tif")
    try:
        with rasterio.open(tif_file) as src1:
            out_image, out_transform = mask(src1, geo_json, crop=True)
            out_meta = src1.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

            # Save cropped image
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)
    except (ValueError, WindowError) as e:
        print(f"Error masking raster: {e}")
        
    print(f"Saved cropped intersection as {output_path}")

In [ ]:
tif_files = glob(os.path.join("risk_layouts", "*.tif"))
for tif_file in tif_files:
    # print(f"Processing {tif_file}...")
    with rasterio.open(tif_file) as src:
        wfpi_data = src.read(1)
    plt.imshow(wfpi_data, cmap='hot', vmin=0, vmax=500)
    plt.colorbar(label="WFPI Value")
    plt.title("Wind-enhanced Fire Potential Index (WFPI)")
    plt.show()

In [ ]:
# move the .npy files from the risk_layouts folder to the appropriate folder
import shutil

# Path to the folder containing .tif files
folder_path = "sim2real_layouts"

# Get a list of all .tif files in the folder
tif_files = glob(os.path.join(folder_path, "*.tif"))

layout_list = []

# Open each .tif file
for tif_file in tif_files:
    id = tif_file.split('/')[1].split('_E')[0]
    rskfile_path = os.path.join("risk_layouts_npy", f"whpi2023_cnt_{id}.npy")
    print("copying", rskfile_path, "to", os.path.join("applicable_layouts", f"{id}/static_risk.npy"))
    #shutil.copy(rskfile_path, os.path.join("applicable_layouts", f"whpi2023_cnt_{id}.npy"))
